In [ ]:
import re
import math as mt 
import pandas as pd
import requests 
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ar=pd.read_csv(r'C:\Users\prave\Downloads\Input.xlsx - Sheet1.csv')
ar.head(3)

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...


In [ ]:
URLS=ar['URL'].to_list()

In [ ]:
# extracting title from wesite
list_title=[] 
for u in URLS:
        reqs = requests.get(u) # using the BeautifulSoup module
        soup = BeautifulSoup(reqs.text, 'html.parser')
        
        for title in soup.find_all('title'):
            list_title.append(title.get_text())

In [ ]:
# Extracting article text from the website

list_text=[]
from bs4 import BeautifulSoup
import requests
for u in URLS:
    
    r=requests.get(u)
    soup=BeautifulSoup(r.content,"html.parser")
    div_text=soup.find("div",{"class":"td-main-content-wrap td-container-wrap"}).get_text()
    list_text.append(div_text)

In [ ]:
ar['title']=list_title
ar['text']=list_text

In [ ]:
ar.head(3)

,URL_ID,URL,title,text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes |...,\n\nHome What We Think AI in healthcare to I...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,\n\nHome What We Think What if the Creation ...
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,\n\nHome What We Think What Jobs Will Robots...


In [ ]:
# remove second title from the  title column
ar['title']=ar['title'].apply(lambda x:x.split('|')[0])

# removing before introduction text i.e(header text)
ar['text']=ar['text'].apply(lambda x:x.split("Introduction\n")[-1])

# removing footer text
ar['text']=ar['text'].apply(lambda x:x.split("\nBlackcoffer Insights")[0])

# some articles are not have "introduction" title and directly written contents. 
# so, for those kind of article we remove header like below
ar['text']=ar['text'].apply(lambda x:x.split('FacebookTwitterPinterestWhatsApp\n\n\n\n\n\n\n')[-1])

In [ ]:
ar.shape

(114, 4)

## Cleaning the StopWords

In [ ]:
df_stop=ar.copy()

In [ ]:
# remove stopwords

file=open(r'C:\users\prave\Downloads\StopWords_GenericLong.txt','r')
generic = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_Generic.txt','r')
gene = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_DatesandNumbers.txt','r')
dates = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_Currencies.txt','r')
currency = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_Auditor.txt','r')
audit = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_Names.txt','r')
names = file.read().split()

file=open(r'C:\users\prave\Downloads\StopWords_Geographic.txt','r')
geo= file.read().split()

In [ ]:
# tokenizing the text column

def tokenization(text):
    text=re.split('\W+',text)
    return text

df_stop['tokenized']=df_stop['text'].apply(lambda x:tokenization(x.lower()))

In [ ]:
def remove_StopWords_GenericLong(text):
    text=[word for word in text if word not in generic]
    return text
df_stop['No_generic']=df_stop['tokenized'].apply(lambda x:remove_StopWords_GenericLong(x))
df_stop.loc[:,['tokenized','No_generic']].head(3)

,tokenized,No_generic
0,"[, if, anything, kills, over, 10, million, peo...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, a, fascination, in, itself, car...","[human, minds, fascination, carrying, potentia..."
2,"[ai, is, rapidly, evolving, in, the, employmen...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_Gene(text):
    text=[word for word in text if word not in gene]
    return text
df_stop['No_gene']=df_stop['No_generic'].apply(lambda x:remove_StopWords_Gene(x))
df_stop.loc[:,['tokenized','No_generic','No_gene']].head(3)

,tokenized,No_generic,No_gene
0,"[, if, anything, kills, over, 10, million, peo...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, a, fascination, in, itself, car...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, is, rapidly, evolving, in, the, employmen...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_dates(text):
    text=[word for word in text if word not in dates]
    return text

df_stop['No_dates']=df_stop['No_gene'].apply(lambda x:remove_StopWords_dates(x))
df_stop.loc[:,['No_generic','No_gene','No_dates']].head(3)

,No_generic,No_gene,No_dates
0,"[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_currency(text):
    text=[word for word in text if word not in currency]
    return text

df_stop['No_currency']=df_stop['No_dates'].apply(lambda x:remove_StopWords_currency(x))
df_stop.loc[:,['No_generic','No_gene','No_currency']].head(3)

,No_generic,No_gene,No_currency
0,"[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_names(text):
    text=[word for word in text if word not in names]
    return text
df_stop['No_names']=df_stop['No_currency'].apply(lambda x:remove_StopWords_names(x))
df_stop.loc[:,['No_generic','No_currency','No_names']].head(3)

,No_generic,No_currency,No_names
0,"[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_geo(text):
    text=[word for word in text if word not in geo]
    return text

df_stop['No_geo']=df_stop['No_names'].apply(lambda x:remove_StopWords_geo(x))
df_stop.loc[:,['No_generic','No_currency','No_geo']].head(3)

,No_generic,No_currency,No_geo
0,"[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
def remove_StopWords_audit(text):
    text=[word for word in text if word not in audit]
    return text

df_stop['No_audit']=df_stop['No_geo'].apply(lambda x:remove_StopWords_audit(x))
df_stop.loc[:,['No_generic','No_currency','No_audit']].head(3)

,No_generic,No_currency,No_audit
0,"[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,"[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
df_stop['text']=df_stop['No_audit']

In [ ]:
df_stop.head(3)

,URL_ID,URL,title,text,tokenized,No_generic,No_gene,No_dates,No_currency,No_names,No_geo,No_audit
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes,"[, kills, 10, million, people, decades, highly...","[, if, anything, kills, over, 10, million, peo...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly...","[, kills, 10, million, people, decades, highly..."
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creator?,"[human, minds, fascination, carrying, potentia...","[human, minds, a, fascination, in, itself, car...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia...","[human, minds, fascination, carrying, potentia..."
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,"[ai, rapidly, evolving, employment, sector, ma...","[ai, is, rapidly, evolving, in, the, employmen...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma...","[ai, rapidly, evolving, employment, sector, ma..."


## Lemmatization

In [ ]:
df_lem=df_stop.loc[:,['text','tokenized']]

In [ ]:
import nltk

ps=nltk.PorterStemmer()
def stemmer(text):
    text=[ps.stem(word) for word in text]
    return text
df_lem['stemmed_porter']=df_lem['text'].apply(lambda x:stemmer(x))

from nltk.stem.snowball import SnowballStemmer
s_stemmer=SnowballStemmer(language='english')
def stemming2(text):
    text=[s_stemmer.stem(word) for word in text]
    return text
df_lem['stemmed_Snowball']=df_lem['text'].apply(lambda x:stemming2(x))

wn=nltk.WordNetLemmatizer()
def lemmatizer(text):
    text=[wn.lemmatize(word) for word in text]
    return text
df_lem['lemmatized']=df_lem['text'].apply(lambda x:lemmatizer(x))


In [ ]:
df_lem['text']=df_lem['lemmatized']

In [ ]:
# Removing Empty string 

df_lem['text'] = df_lem['text'].apply(lambda x:[i for i in x if i])

In [ ]:
df_lem.head(3)

,text,tokenized,stemmed_porter,stemmed_Snowball,lemmatized
0,"[kill, 10, million, people, decade, highly, in...","[, if, anything, kills, over, 10, million, peo...","[, kill, 10, million, peopl, decad, highli, in...","[, kill, 10, million, peopl, decad, high, infe...","[, kill, 10, million, people, decade, highly, ..."
1,"[human, mind, fascination, carrying, potential...","[human, minds, a, fascination, in, itself, car...","[human, mind, fascin, carri, potenti, tinker, ...","[human, mind, fascin, carri, potenti, tinker, ...","[human, mind, fascination, carrying, potential..."
2,"[ai, rapidly, evolving, employment, sector, ma...","[ai, is, rapidly, evolving, in, the, employmen...","[ai, rapidli, evolv, employ, sector, matter, i...","[ai, rapid, evolv, employ, sector, matter, inv...","[ai, rapidly, evolving, employment, sector, ma..."


## Sentiment Scores

In [ ]:
df_scores=df_lem.loc[:,['text']]

In [ ]:
file = open(r'C:\users\prave\Downloads\negative-words.txt', 'r')
neg_words = file.read().split()
file = open(r'C:\users\prave\Downloads\positive-words.txt', 'r')
pos_words = file.read().split()

In [ ]:
df_scores.head(3)

,text
0,"[kill, 10, million, people, decade, highly, in..."
1,"[human, mind, fascination, carrying, potential..."
2,"[ai, rapidly, evolving, employment, sector, ma..."


In [ ]:
# Positive Score is calculated by assigning the value of +1 for each word if found in the Positive Dictionary
# and then adding up all the values.
num_pos = df_scores['text'].map(lambda x: len([i for i in x if i in pos_words]))
df_scores['pos_score'] = num_pos+1

# Negative Score is calculated by assigning the value of -1 for each word if found in the Negative Dictionary 
# and then adding up all the values. We multiply the score with -1 so that the score is a positive number.
num_neg = df_scores['text'].map(lambda x: len([i for i in x if i in neg_words]))
df_scores['neg_score'] = num_neg-1*-1

In [ ]:
df_scores.loc[:,['text','pos_score','neg_score']].head(3)

,text,pos_score,neg_score
0,"[kill, 10, million, people, decade, highly, in...",81,38
1,"[human, mind, fascination, carrying, potential...",78,39
2,"[ai, rapidly, evolving, employment, sector, ma...",70,43


In [ ]:
# converting list string into text

df_scores['text']= df_scores['text'].apply(' '.join)

In [ ]:
df_scores.head()

,text,pos_score,neg_score
0,kill 10 million people decade highly infectiou...,81,38
1,human mind fascination carrying potential tink...,78,39
2,ai rapidly evolving employment sector matter i...,70,43
3,give rise smarter human intelligence form arti...,75,33
4,machine intelligence invention humanity make n...,68,30


In [ ]:
# Total words after cleaning

df_scores['total_clean_words'] = df_scores['text'].apply(lambda x: len(x.split()))

In [ ]:
# Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)

df_scores['polarity_score']= (df_scores['pos_score'] - df_scores['neg_score'])/ ((df_scores['pos_score'] + df_scores['neg_score']) + 0.000001)

In [ ]:
# Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)

df_scores['sub_score']=(df_scores['pos_score']+df_scores['neg_score'])/(df_scores['total_clean_words']+ 0.000001)

In [ ]:
df_scores.head(3)

,text,pos_score,neg_score,total_clean_words,polarity_score,sub_score
0,kill 10 million people decade highly infectiou...,81,38,1037,0.361345,0.114754
1,human mind fascination carrying potential tink...,78,39,614,0.333333,0.190554
2,ai rapidly evolving employment sector matter i...,70,43,867,0.238938,0.130334


## Analysis of Readability

In [ ]:
df_word=df_scores.copy()

In [ ]:
# No of words before clean

df_word['no_of_words_before_clean']=ar['text'].apply(lambda x: len(x.split()))

In [ ]:
# sentence count

import textstat

df_word['sentence_count'] = ar['text'].apply(textstat.sentence_count)

In [ ]:
# count complex words

def count_complex_words(text):
    words = text.split()
    complex_words = [word for word in words if textstat.syllable_count(word) > 2]
    return len(complex_words)

df_word['complex_words']=ar['text'].apply(lambda x:count_complex_words(x))

In [ ]:
# counting syllable words 

def syllables_count(word):
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    word=word.lower()
    vowels = 'aeiouy'
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('es'):
        count-=1
    if word.endswith('ed'):
        count-=1
    if count == 0:
        count += 1
    return count
df_word['syllable_words']=ar['text'].apply(lambda x:syllables_count(x))

In [ ]:
# count Syllables per word

import pyphen
# create a Pyphen object for the English language
dic = pyphen.Pyphen(lang='en')


def count_syllables(word):
    return len(dic.inserted(word).split('-'))

df_word['syllables_per_word'] = ar['text'].apply(lambda x: sum([count_syllables(word) for word in x.lower().split()]))

In [ ]:
# Personal Pronouns

import re
pattern = re.compile(r'\b(?:I|we|my|ours|us)\b', flags=re.IGNORECASE)

df_word['personal pronouns'] = [pattern.findall(text) for text in ar['text']]

In [ ]:
# Average number of words per sentence

sentences = ar['text'].apply(nltk.sent_tokenize)

word_counts = sentences.apply(lambda x: [len(nltk.word_tokenize(sentence)) for sentence in x])

total_words = word_counts.sum()
total_sentences = word_counts.apply(len).sum()

avg_words_per_sentence  = total_words / total_sentences

df_word['avg_no_of_words_per_Sentence']=df_word['no_of_words_before_clean']/ df_word['sentence_count']

In [ ]:
# Average sentence length
import numpy as np
def get_avg_sentence_length(text):
    sentences = nltk.sent_tokenize(text)
    lengths = [len(sent.split()) for sent in sentences]
    return np.mean(lengths)

df_word['avg_sent_length'] = ar['text'].apply(get_avg_sentence_length)

In [ ]:
# Percentage of Complex words = the number of complex words / the number of words 

df_word['%_of_complex_words']=df_word['complex_words']/df_word['no_of_words_before_clean']

In [ ]:
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

df_word['fog_index']=0.4 * (df_word['avg_sent_length'] + df_word['%_of_complex_words'])

In [ ]:
# Average number of words

df_word['char_count'] = ar['text'].str.len()

df_word['avg_word_length'] = df_word['char_count'] / df_word['no_of_words_before_clean']

### Final Output Structure

In [ ]:
df_word.loc[:,['URL_ID','URL']]=ar.loc[:,['URL_ID','URL']]

In [ ]:
df_word=df_word.loc[:,['URL_ID','URL','pos_score','neg_score','polarity_score','sub_score','avg_sent_length','%_of_complex_words',
              'fog_index','avg_no_of_words_per_Sentence','complex_words','total_clean_words','syllables_per_word',
              'personal pronouns','avg_word_length']]

In [ ]:
# rename columns names
df_word.rename(columns={'URL_ID':'URL_ID','URL':'URL','pos_score':'POSITIVE SCORE','neg_score':'NEGATIVE SCORE','polarity_score'
                       'sub_score':'SUBJECTIVITY SCORE','avg_sent_length':'AVERAGE SENTENCE LENGTH','%_of_complex_words':'PERCENTAGE OF COMPLEX WORDS',
                       'fog_index':'FOG INDEX','avg_no_of_words_per_Sentence':'AVERAGE NUMBER OF WORDS PER SENTENCE','complex_words':'COMPLEX WORDS',
                       'total_clean_words':'WORD COUNT','syllables_per_word':'SYLLABLES PER WORD','personal pronouns':'PERSONAL PRONOUNS',
                       'avg_word_length':'AVERAGE WORD LENGTH'},inplace=True)

In [ ]:
df_word.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,polarity_score,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE NUMBER OF WORDS PER SENTENCE,COMPLEX WORDS,WORD COUNT,SYLLABLES PER WORD,PERSONAL PRONOUNS,AVERAGE WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,81,38,0.361345,0.114754,23.394737,0.207537,9.440909,22.506329,369,1037,3070,[us],6.793026
1,38,https://insights.blackcoffer.com/what-if-the-c...,78,39,0.333333,0.190554,17.734177,0.133476,7.147061,18.194805,187,614,2064,"[we, us, us, i, us, we, we]",5.997859
2,39,https://insights.blackcoffer.com/what-jobs-wil...,70,43,0.238938,0.130334,20.083333,0.200356,8.113476,20.083333,338,867,2790,"[We, us, us]",6.529935
3,40,https://insights.blackcoffer.com/will-machine-...,75,33,0.388889,0.151685,17.414894,0.139890,7.021913,17.602151,229,712,2399,"[us, we, us, we, we, we, we, we, we, we, US, w...",5.981063
4,41,https://insights.blackcoffer.com/will-ai-repla...,68,30,0.387755,0.116114,22.102564,0.157773,8.904135,20.282353,272,844,2691,"[us, We, we, we, we, we, us, us, we, we, We, W...",6.255800
